# Building conversational AI with the Rasa stack
![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaX3LNhGcAe1HnPZSuWS0oH6af0LJHXcH7If1sQgLCFAT1chNGFg)


This notebook is a basis for my workshop at PyData 2018 Berlin. If you have any questions or would like to learn more about anything included in this notebook, please let me know or get in touch by juste@rasa.com.

In this workshop we are going to build a chatbot capable of checking in on people's mood and take the necessary actions to cheer them up. 


The tutorial consists of three parts:


*   Part 0: Installation and setup
*   Part 1: Teaching the chatbot to understand user inputs using Rasa NLU model
*   Part 2: Teaching the chatbot to handle multi-turn conversations using dialogue management model.
*   Part 3: Resources and tips

## Part 0: Installation

### Let's start with jupyter configuration

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Installation of Rasa
Let's start with the installation of Rasa NLU, Rasa Core and a spacy language model. If you have already installed, you can skip this step. 

In [3]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -U rasa_core rasa_nlu[spacy];

# as well as install a language model:
!{python} -m spacy download en_core_web_md
!{python} -m spacy link en_core_web_md en --force;

    100% |████████████████████████████████| 215kB 4.7MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/5b/b7/e1211e256172284998fc0d86abb117e54110be54d646e3c7a3fadec6d0d0/rasa_nlu-0.14.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 51kB 9.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/b6/8c/20be00d1e30bd3d905e2977ba0115822985d159196194e5dc58ddf127e07/fbmessenger-5.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4a/bd/eee1157fc2d8514970b345d69cb9975dcd1e42cd7e61146ed841f6e68309/typing-3.6.6-py3-none-any.whl
    100% |████████████████████████████████| 112kB 7.6MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/64/bd/2756ddf350c4bb308e3255f9dcd6610f8b01344947bf74d5d166dc66b0a2/fakeredis-0.10.3-py2.py3-none-any.whl
    100% |████████████████████████████████| 409kB 3.3MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/65/cb/683f71ff8daa3aea0a5cbb276074de

    100% |████████████████████████████████| 83.1MB 73kB/s  eta 0:00:01-1 day, 23:59:525                     | 4.8MB 7.7MB/s eta 0:00:11    6% |██                              | 5.3MB 15.9MB/s eta 0:00:05    9% |███                             | 7.9MB 7.6MB/s eta 0:00:10    12% |████                            | 10.2MB 11.6MB/s eta 0:00:07    12% |████                            | 10.6MB 6.1MB/s eta 0:00:12    17% |█████▋                          | 14.6MB 9.4MB/s eta 0:00:08    17% |█████▊                          | 14.9MB 13.2MB/s eta 0:00:06    21% |███████                         | 18.0MB 5.1MB/s eta 0:00:13    27% |████████▋                       | 22.5MB 7.7MB/s eta 0:00:08    29% |█████████▍                      | 24.3MB 4.9MB/s eta 0:00:12    40% |████████████▉                   | 33.3MB 19.7MB/s eta 0:00:03    67% |█████████████████████▋          | 56.1MB 10.9MB/s eta 0:00:03    69% |██████████████████████▍         | 58.1MB 9.5MB/s eta 0:00:03    76% |████████████████████████▌  

    100% |████████████████████████████████| 204kB 9.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/77/61/ae928ce6ab85d4479ea198488cf5ffa371bd4ece2030c0ee85ff668deac5/ConfigArgParse-0.13.0.tar.gz
    100% |████████████████████████████████| 51kB 9.3MB/s eta 0:00:01
    100% |████████████████████████████████| 10.6MB 1.9MB/s ta 0:00:011   13% |████▍                           | 1.5MB 8.9MB/s eta 0:00:02    27% |████████▋                       | 2.9MB 8.2MB/s eta 0:00:01    64% |████████████████████▊           | 6.9MB 7.6MB/s eta 0:00:01    69% |██████████████████████▏         | 7.3MB 9.2MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 15.2MB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 14.5MB/s a 0:00:01
    100% |████████████████████████████████| 3.1MB 4.4MB/s ta 0:00:011    10% |███▍                            | 317kB 11.5MB/s eta 0:00:01    27% |█████████                       | 849kB 10.4MB/s eta 0:00:01    62% |██████████████

    100% |████████████████████████████████| 61kB 6.5MB/s ta 0:00:01
    100% |████████████████████████████████| 3.1MB 2.6MB/s ta 0:00:011
    100% |████████████████████████████████| 757kB 7.7MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 3.2MB/s ta 0:00:011
    100% |████████████████████████████████| 1.9MB 5.5MB/s ta 0:00:011
    100% |████████████████████████████████| 614kB 8.4MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 5.6MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 6.5MB/s ta 0:00:011
    100% |████████████████████████████████| 92kB 9.3MB/s eta 0:00:01
    100% |████████████████████████████████| 552kB 8.7MB/s eta 0:00:01
    100% |████████████████████████████████| 174kB 5.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 8.8MB/s eta 0:00:01
    100% |████████████████████████████████| 317kB 15.1MB/s ta 0:00:01


  Running setup.py bdist_wheel for terminaltables ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
  Running setup.py bdist_wheel for colorclass ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/d1/86/9d/16127127306a92d7fd30267890a5634026c045391979c4c317
  Running setup.py bdist_wheel for webexteamssdk ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/98/09/8a/03b3fcfe0a351b960427e278d87191e9a3065cd2a36b84ab3d
  Running setup.py bdist_wheel for flask-jwt-simple ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/5d/86/c2/49573c0ce194f3073438e56dad49c462fe9c9a69a4fbe7b3fc
  Running setup.py bdist_wheel for simplejson ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7

    100% |████████████████████████████████| 120.9MB 26.1MB/s ta 0:00:01-1 day, 23:59:59                    | 133kB 331kB/s eta 0:06:05    4% |█▌                              | 5.5MB 8.1MB/s eta 0:00:15    4% |█▋                              | 5.9MB 6.2MB/s eta 0:00:19    5% |█▉                              | 6.7MB 13.9MB/s eta 0:00:09    6% |██                              | 7.7MB 1.6MB/s eta 0:01:09    6% |██                              | 7.9MB 10.5MB/s eta 0:00:11    6% |██▏                             | 8.1MB 2.4MB/s eta 0:00:48    7% |██▎                             | 8.7MB 6.0MB/s eta 0:00:19    7% |██▍                             | 9.0MB 1.6MB/s eta 0:01:11    7% |██▌                             | 9.4MB 4.0MB/s eta 0:00:29    7% |██▌                             | 9.6MB 582kB/s eta 0:03:12    9% |███                             | 11.1MB 7.8MB/s eta 0:00:15    10% |███▌                            | 13.1MB 3.3MB/s eta 0:00:33    11% |███▉                            | 14.4MB 4.0MB/s

Let's test the installation - we should have rasa_nlu: 0.12.3 and rasa_core: 0.9.6 installed, and spacy model should be available.

In [4]:
import rasa_nlu
import rasa_core
import spacy

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))
print("Loading spaCy language model...")
print(spacy.load("en")("Hello world!"))

rasa_nlu: 0.14.1 rasa_core: 0.13.0
Loading spaCy language model...
Hello world!


### Some additional Tools needed
To do some of the visualizations you will also need graphviz. If you don't have graphviz installed, and this doesn't work: don't worry. I'll show you the graph and besides that visualization everything else will work.

Try installing with anyone of these (or adapt to your operating system):

In [7]:
!sudo apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!brew install graphviz

[sudo] password for jovyan: 


and another python package and we are ready to go:

In [8]:
!{python} -m pip install pygraphviz;

    100% |████████████████████████████████| 122kB 1.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for pygraphviz ... error
  Complete output from command /opt/conda/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-mv067g0g/pygraphviz/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-sa6p4kdf --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.6
  creating build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/agraph.py -> build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/release.py -> build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/version.py -> build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/__init__.py -> build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/graphviz.py -> build/lib.linux-x86_64-3.6/pygraphviz
  creating build

## Part 1: Natural Language Understanding

At first, let's teach our chatbot how to understand user inputs. To do that, we are going to build a Rasa NLU model. Here is some data to get started: 

**Conversation_1:**   
U: Hello  
B: Hello, how are you doing?  
U: I am doing great!  
B: Great. Carry on!  
    
**Conversation_2:**  
U: Hey  
B: Hello, how are you doing?  
U: I am very sad  
B: To cheer you up, I can show you a cute picture of a cat, a dog or a bird. Choose one :)  
U: A kitten  
B: Here is something to cheer you up. Did that help?  
U: Yes  
B: Goodbye  
  
**Conversation_3:**    
U: Heya  
B: Hello, how are you doing?  
U: Not so good and the only thing that could help me feel better is a picture of a puppy  
B: Here is something to cheer you up. Did that help?  
U: No  
B: Goodbye  
U: Bye  

### Creating the training data for language understanding model


Lets create some training data here, grouping user messages by their `intents`. The intent describes what the messages *mean*. Another important part of training data are `entities` - pieces of information which help a chatbot understand what specifically a user is asking about. Entities are labeled using the markdown link syntex: `[entity value](entity_type)` [More information about the data format](https://nlu.rasa.com/dataformat.html#markdown-format).

In [1]:
nlu_md = """
## intent:greet
- hey
- hello there


## intent:goodbye
- cu
- good by
- cee you later

"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


### Defining the NLU model

Once the training data is ready, we can define our NLU model. We can do that by constructing the processing pipeline which defines how structured data is extracted from unstructured user inputs. 

In [2]:
config = """
language: "en"

pipeline:
- name: "sentiment.MyComponentA"
- name: "sentiment.MyComponentB"

""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


### Training the Rasa NLU Model

We're going to train a model to recognise user inputs, so that when you send a message like "hello" to your bot, it will recognise this as a `"greet"` intent.

In [ ]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 5 (2 distinct intents)
	- Found intents: 'goodbye', 'greet'
	- entity examples: 0 (0 distinct entities)
	- found entities: 



A
> /home/jovyan/work/sentiment.py(46)__init__()
-> super().__init__(component_config)
(Pdb) c
> /home/jovyan/work/sentiment.py(138)__init__()
-> super().__init__(component_config)
(Pdb) c


INFO:rasa_nlu.model:Starting to train component CompA


A
> /home/jovyan/work/sentiment.py(62)train()
-> pass
(Pdb) training_data
(Pdb) cfg
(Pdb) kwargs
{}
(Pdb) dir(training_data)
['MIN_EXAMPLES_PER_ENTITY', 'MIN_EXAMPLES_PER_INTENT', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_lazy_entities', '_lazy_entity_examples', '_lazy_examples_per_entity', '_lazy_examples_per_intent', '_lazy_intent_examples', '_lazy_intents', 'as_json', 'as_markdown', 'entities', 'entity_examples', 'entity_synonyms', 'examples_per_entity', 'examples_per_intent', 'intent_examples', 'intents', 'lookup_tables', 'merge', 'persist', 'print_stats', 'regex_features', 'sanitize_examples', 'sort_regex_features', 'sorted_entities', 'sorted_intent_examples', 'trai

INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component CompB


> /home/jovyan/work/sentiment.py(153)train()
-> with the training data. The component can rely on
(Pdb) list
148  	
149  	    def train(self, training_data, cfg, **kwargs):
150  	        """Train this component.
151  	
152  	        This is the components chance to train itself provided
153  ->	        with the training data. The component can rely on
154  	        any context attribute to be present, that gets created
155  	        by a call to :meth:`components.Component.pipeline_init`
156  	        of ANY component and
157  	        on any context attributes created by a call to
158  	        :meth:`components.Component.train`
(Pdb) traing_data
*** NameError: name 'traing_data' is not defined
(Pdb) training_data
(Pdb) training_data.training_example[0].text
*** AttributeError: 'TrainingData' object has no attribute 'training_example'
(Pdb) traing_data
*** NameError: name 'traing_data' is not defined
(Pdb) training_data
(Pdb) training_data.training_examples
[<rasa_nlu.training_data.me

INFO:rasa_nlu.model:Finished training component.


A
> /home/jovyan/work/sentiment.py(85)persist()
-> import pdb;
(Pdb) list
 80  	        of ANY component and
 81  	        on any context attributes created by a call to
 82  	        :meth:`components.Component.process`
 83  	        of components previous to this one."""
 84  	        print('A')
 85  ->	        import pdb;
 86  	        pdb.set_trace()
 87  	        pass
 88  	
 89  	    def persist(self, model_dir):
 90  	        """Persist this component to disk for future loading."""


### Using & evaluating the NLU model

Let's see how the model is performing on some of the inputs:

In [ ]:
pprint(interpreter.parse(""))

Instead of evaluating it by hand, the model can also be evaluated on a test data set (though for simplicity we are going to use the same for test and train):

In [ ]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("nlu.md", model_directory)

# Part 2: Handling the dialogue

We have taught our chatbot how to understand user inputs. Now, it's time to teach our chatbot how to make responses by training a dialogue management model using Rasa Core.

### Writing Stories

The training data for dialogue management models is called `stories`. A story is an actual conversation where user inputs are expressed as intents as well as corresponding entities, and chatbot responses are expressed as actions.


Let's take a look into the format of the stories in more detail:

A story starts with `##` and you can give it a name. 
Lines that start with `*` are messages sent by the user. Although you don't write the *actual* message, but rather the intent (and the entities) that represent what the user *means*. 
Lines that start with `-` are *actions* taken by your bot. In this case all of our actions are just messages sent back to the user, like `utter_greet`, but in general an action can do anything, including calling an API and interacting with the outside world. 

In [ ]:
stories_md = """


"""

%store stories_md > stories.md

### Defining a Domain

The domain specifies the universe that the bot operates in. In chatbot's world this universe consists of intents and entities as well as the actions which appear in training stories. The domain can also contain the templates for the answers a chabot should use to respond to the user and slots which will help the chatbot to keep track of the context. Let's look into the domain of our bot:

In [ ]:
domain_yml = """
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy
- inform

    
entities:
- group

actions:
- utter_greet
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_unclear
- utter_ask_picture


templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  
  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
  
  utter_ask_picture:
  - text: "To cheer you up, I can show you a cute picture of a dog, a cat or a bird. Which one do you choose?"
"""

%store domain_yml > domain.yml

### Adding Custom Actions

The responses of the chatbot can be more than just simple text responses - we can call an API to retrieve some data which can later be used to create a response to user input. Let's create a custom action for our bot which, when predicted, will make an API and retrieve a picture of a dog, a cat or a bird, depending on which was specified by the user. The bot will know which type of picture should be received by retrieving the value of the slot `group`.


In [ ]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.display import Image

import requests

class ApiAction(Action):
    def name(self):
        return "action_retrieve_image"

    def run(self, dispatcher, tracker, domain):
        

        group = 
        r = 
        
        response = r.content.decode()
        response = response.replace('["',"")
        response = response.replace('"]',"")
        
        dispatcher.utter_message("")

### Pro Tip: Visualising the Training Data

You can visualise the stories to get a sense of how the conversations go. This is usually a good way to see if there are any stories which don't make sense


In [ ]:
from IPython.display import Image
from rasa_core.agent import Agent

agent = Agent('domain.yml')
agent.visualize("stories.md", "story_graph.html", max_history=2)

### Training your Dialogue Model

Now we are good to train the dialogue management model. We can specify what policies should be used to train it - in this case, the model is a neural network implemented in Keras which learns to predict which action to take next. We can also tweak the parameters of what percentage of training examples should be used for validation and how many epochs should be used for training.

In [ ]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions


agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=200
)

agent.persist('models/dialogue')

### Starting up the bot (with NLU)

Now it's time for the fun part - starting the agent and chatting with it. We are going to start the `Agent` by loading our just trained dialogue model and using the previously trained nlu model as an interpreter for incoming user inputs.

In [ ]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

### Talking to the Bot (with NLU)

Let's have a chat!

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])


### Evaluation of the dialogue model
As with the NLU model, instead of just subjectively testing the model, we can also evaluate the model on a dataset. You'll be using the training data set again, but usually you'd use a test data set separate from the training data.

In [ ]:
from rasa_core.evaluate import run_story_evaluation

run_story_evaluation("stories.md", "models/dialogue", 
                     nlu_model_path=None, 
                     max_stories=None, 
                     out_file_plot="story_eval.pdf")

### Interactive learning
Unfortunately, this doesn't work in Jupyter yet. Hence, we going to do this on the command line. To start the interactive training session open your command line and run `train_online.py` script.

### Resources and tips

- Rasa NLU [documentation](https://nlu.rasa.com/)
- Rasa Core [documentation](https://core.rasa.com/)
- Rasa Community on [Gitter](https://gitter.im/RasaHQ/home)
- Rasa [Blog](https://blog.rasa.com/)